[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/intel/e2eAIOK/blob/main/demo/ma/domain_adapter/Model_Adapter_Domain_Adapter_Walkthrough_Unet_KITS19.ipynb)

# Model Adapter Domain Adapter Walkthrough Demo

Model Adapter is a convenient framework can be used to reduce training and inference time, or data labeling cost by efficiently utilizing public advanced models and those datasets from many domains. It mainly contains three components served for different cases: Finetuner, Distiller, and Domain Adapter. 

Directly applying pre-trained model into target domain cannot always work due to covariate shift and label shift, while fine-tuning is also not working due to the expensive labeling in some domains. Even if users invest resource in labeling, it will be time-consuming and delays the model deployment.

In this demo, we will introduce how to use domain adaptation to transfer knowledge in medical image semantic segmentation. 

Unlike the [built-in demo](./Model_Adapter_Domain_Adapter_builtin_Unet_KITS19.ipynb), we will illustrate how to invoke the Model Adaptor API on your own workflow.

# Content

* [Domain Adapter Overview](#Domain-Adapter-Overview)
* [1. Environment Setup](#1.-Environment-Setup)
* [2. Task Description](#2.-Task-Description)
* [3. Data Preparation](#3.-Data-Preparation)
* [4. Data Preprocessing](#4.-Data-Preprocessing)
    * [4.1 Data Alignment](#4.1-Data-Alignment)
    * [4.2 Data Verification](#4.2-Data-Verification)
    * [4.3 Data Target Spacing Sample & Normalization](#4.3-Data-Target-Spacing-Sample-&-Normalization)
* [5. Launch training](#5.-Launch-training)
    * [5.1 Pre-train Target Domain](#5.1-Pre-train-Target-Domain)
    * [5.2 Domain Adaption from AMOS to KiTS](#5.2-Domain-Adaption-from-AMOS-to-KiTS)
* [6. Launch Inference & Evaluation](#6.-Launch-Inference-&-Evaluation)
    * [6.1 Inference on KiTS Dataset with Adapted Model](#6.1-Inference-on-KiTS-Dataset-with-Adapted-Model)
    * [6.2 Evaluate the Prediction on KiTS Using the Given Label](#6.2-Evaluate-the-Prediction-on-KiTS-Using-the-Given-Label)  
* [7. Visualization of Data and Segmentations](#7.-Visualization-of-Data-and-Segmentations)  

# Domain Adapter Overview

Adapter aims at reusing the transferable knowledge with the help of another labeled dataset with same learning task. That is, achieving better generalization with little labeled target dataset or achieving a competitive performance in label-free target dataset.

The following picture show the network strcture of domain adaption, which add a discriminator to users' base network, and try to differentiate the souce domain data and target domain data, hence, it can force the feature extractor to learn a generalized feature representation among domains.

![Adapter](../imgs/adapter.png)

## 1. Environment Setup

### (Option 1) Use Pip install - recommend
```bash
pip install e2eAIOK-ModelAdapter --pre
```

### (Option 2) Use Docker 

Step1. prepare code
   ``` bash
   git clone https://github.com/intel/e2eAIOK.git
   cd e2eAIOK
   git submodule update --init –recursive
   ```
    
Step2. build docker image
   ``` bash
   python3 scripts/start_e2eaiok_docker.py -b pytorch112 --dataset_path ${dataset_path} -w ${host0} ${host1} ${host2} ${host3} --proxy  "http://addr:ip"
   ```
   
Step3. run docker and start conda env
   ``` bash
   sshpass -p docker ssh ${host0} -p 12347
   conda activate pytorch-1.12.0
   ```
  
Step4. Start the jupyter notebook and tensorboard service
   ``` bash
   nohup jupyter notebook --notebook-dir=/home/vmagent/app/e2eaiok --ip=${hostname} --port=8899 --allow-root &
   nohup tensorboard --logdir /home/vmagent/app/data/tensorboard --host=${hostname} --port=6006 & 
   ```
   Now you can visit demso in `http://${hostname}:8899/`, and see tensorboad log in ` http://${hostname}:6006`.

# 2. Task Description

* Our source domain is AMOS dataset(Download AMOS data from [here](https://amos22.grand-challenge.org/Dataset/)), which provides 500 CT and 100 MRI scans with voxel-level annotations of 15 abdominal organs, including the spleen, right kidney, left kidney, gallbladder, esophagus, liver, stomach, aorta, inferior vena cava, pancreas, right adrenal gland, left adrenal gland, duodenum, bladder, prostate/uterus.
* Our target domain is KiTS dataset(Download KiTS data from [here](https://github.com/neheller/kits19)), which provides 300 CT scans with voxel-level annotations of kidney organs and kidney tumor.

* Our task is to explore reliable kidney semantic segmentation methodologies with the help of labeled AMOS dataset and unlabeled KiTS dataset, evalutaion metric is kidney dice score in target domain.

# 3. Data Preparation

- Download KiTS data from [here](https://github.com/neheller/kits19)
- Download AMOS data from [here](https://amos22.grand-challenge.org/Dataset/)
* Remember to put all your data in right places, now your files should be located at:
   - Images at: ```${nnUNet_raw_data_base}/nnUNet_raw_data/Task507_KiTS_kidney/imagesTr/```
   - Labels/Segmentations at: ```${nnUNet_raw_data_base}/nnUNet_raw_data/Task507_KiTS_kidney/labelsTr/```
   - Please refer to [here](https://github.com/MIC-DKFZ/nnUNet) to know how to put all your data in your `${dataset_path}` in right format.

- Then, setup some enviroment variables
    - It tell the program where to read data, and where to write the output model and log

In [ ]:
import os
os.environ['nnUNet_raw_data_base'] = "/home/vmagent/app/data/nnUNet_raw_data_base" 
os.environ['nnUNet_preprocessed'] = "/home/vmagent/app/data/nnUNet_preprocessed"
os.environ['RESULTS_FOLDER'] = "/home/vmagent/app/data/nnUNet_trained_models"

- Then, We put the data in `$nnUNet_raw_data_base/nnUNet_raw_data`, the structure should look like (*for simlicy, we only take 5 case of each task for demostration*):

In [ ]:
!tree $nnUNet_raw_data_base/nnUNet_raw_data

/home/vmagent/app/dataset/nnUNet_raw_data_base/nnUNet_raw_data
├── Task041_KiTS
│   ├── dataset.json
│   ├── imagesTr
│   │   ├── case_00000_0000.nii.gz
│   │   ├── case_00001_0000.nii.gz
│   │   ├── case_00002_0000.nii.gz
│   │   ├── case_00003_0000.nii.gz
│   │   └── case_00004_0000.nii.gz
│   └── labelsTr
│       ├── case_00000.nii.gz
│       ├── case_00001.nii.gz
│       ├── case_00002.nii.gz
│       ├── case_00003.nii.gz
│       └── case_00004.nii.gz
└── Task505_AMOS
    ├── imagesTr
    │   ├── amos_0001.nii.gz
    │   ├── amos_0004.nii.gz
    │   ├── amos_0005.nii.gz
    │   ├── amos_0006.nii.gz
    │   └── amos_0007.nii.gz
    ├── labelsTr
    │   ├── amos_0001.nii.gz
    │   ├── amos_0004.nii.gz
    │   ├── amos_0005.nii.gz
    │   ├── amos_0006.nii.gz
    │   └── amos_0007.nii.gz
    └── task1_dataset.json

6 directories, 22 files


# 4. Data Preprocessing

## 4.1 Data Alignment

- In this part, we do the following thing:
    - Keep the both data in the same axis ordering, for background knowledge, you can refer to [here](https://www.jarvis73.com/2019/06/24/Medical-Imaging-Guide/#13-%E5%9D%90%E6%A0%87%E7%B3%BB%E7%BB%9F)
        - Axis ordering: it determines in what direction we see the medical image, it is adjustable, and something like rotation in natural images, we should make the two dataset have same perspective;
    - Change the tumor annotation in KiTS to kidney, because we cannot know the tumor from source domain AMOS

In [ ]:
%cd /home/vmagent/app/e2eaiok/modelzoo/unet/nnUNet/nnunet

/work/AIDK/AIDK/TransferLearningKit/src/task/medical_segmentation/nnunet


In [ ]:
!python dataset_conversion/amos_convert_label.py

In [ ]:
!python dataset_conversion/kits_convert_label.py basic

/home/vmagent/app/dataset/nnUNet_raw_data_base/nnUNet_raw_data/Task507_KiTS_kidney/labelsTr
case_00003.nii.gz
case_00000.nii.gz
case_00001.nii.gz
case_00002.nii.gz
case_00004.nii.gz
case_00002_0000.nii.gz
('R', 'A', 'S')
case_00003_0000.nii.gz
('R', 'A', 'S')
case_00001_0000.nii.gz
('R', 'A', 'S')
case_00000_0000.nii.gz
('R', 'A', 'S')
case_00004_0000.nii.gz
('R', 'A', 'S')


In [ ]:
# now your raw dataset directory structure should look like:
!tree $nnUNet_raw_data_base/nnUNet_raw_data

/home/vmagent/app/dataset/nnUNet_raw_data_base/nnUNet_raw_data
├── Task041_KiTS
│   ├── dataset.json
│   ├── imagesTr
│   │   ├── case_00000_0000.nii.gz
│   │   ├── case_00001_0000.nii.gz
│   │   ├── case_00002_0000.nii.gz
│   │   ├── case_00003_0000.nii.gz
│   │   └── case_00004_0000.nii.gz
│   └── labelsTr
│       ├── case_00000.nii.gz
│       ├── case_00001.nii.gz
│       ├── case_00002.nii.gz
│       ├── case_00003.nii.gz
│       └── case_00004.nii.gz
├── Task505_AMOS
│   ├── imagesTr
│   │   ├── amos_0001.nii.gz
│   │   ├── amos_0004.nii.gz
│   │   ├── amos_0005.nii.gz
│   │   ├── amos_0006.nii.gz
│   │   └── amos_0007.nii.gz
│   ├── labelsTr
│   │   ├── amos_0001.nii.gz
│   │   ├── amos_0004.nii.gz
│   │   ├── amos_0005.nii.gz
│   │   ├── amos_0006.nii.gz
│   │   └── amos_0007.nii.gz
│   └── task1_dataset.json
├── Task507_KiTS_kidney
│   ├── dataset.json
│   ├── imagesTr
│   │   ├── case_00000_0000.nii.gz
│   │   ├── case_00001_0000.nii.gz
│   │   ├── case_00002_0000.nii.gz
│   │

## 4.2 Data Verification

- Before going any further, verify that the data is present and labels and data matches.

In [ ]:
!nnUNet_plan_and_preprocess -t 507 --verify_dataset_integrity

Verifying training set
checking case case_00000
checking case case_00001
checking case case_00002
checking case case_00003
checking case case_00004
Verifying label values
Expected label values are [0, 1]
Labels OK
Dataset OK


In [ ]:
!nnUNet_plan_and_preprocess -t 508 --verify_dataset_integrity

Verifying training set
checking case amos_0001
checking case amos_0004
checking case amos_0005
checking case amos_0006
checking case amos_0007
Verifying label values
Expected label values are [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
Labels OK
Dataset OK


## 4.3 Data Target Spacing Sample & Normalization

- We need to perform same target spacing sample && normalization in both domains, and saves it into the "nnUNet_preprocessed" folder.

    - Voxel Spacing: it is the distance between voxels, it influence the image size, we can understand it as the resolution in natural images. Every image have different voxel spacing even if they are in the exact one dataset, it is not suitable for convolution operations according to literature, so we usually doing some resampling operations to make the voxel spacing is same in every image of both dataset;

    - Intensity: it is the float value of every pixel in each slice of the grey CT images, usually same organs have similar intensity distribution even if they are captured by different scanners. Currently we use the intensity mean and std from the foreground of source domain dataset to perform normalization in both datasets, since foreground of source dataset have more classes, and we need to segmentation target dataset to these classes, so target dataset executed the same normalization.
    
- So we first process the target domain, get the dataset characteristic, and then apply it to the source domain
- Also some rule based parameters will be extracted in this step, such as model architecture, learning rate, batch size...

In [ ]:
!nnUNet_plan_and_preprocess -t 508 -pl2d None -pl3d ExperimentPlanner3D_v21_customTargetSpacing_kits19

amos_0001
amos_0004
amos_0005
amos_0006
amos_0007
before crop: (1, 78, 512, 512) after crop: (1, 78, 512, 512) spacing: [5.         0.78200001 0.78200001] 

before crop: (1, 99, 512, 512) after crop: (1, 99, 512, 512) spacing: [5.         0.83499998 0.83499998] 

before crop: (1, 80, 768, 768) after crop: (1, 80, 768, 768) spacing: [5.         0.56901044 0.56901044] 

before crop: (1, 90, 768, 768) after crop: (1, 90, 768, 768) spacing: [5.        0.5703125 0.5703125] 

before crop: (1, 107, 768, 768) after crop: (1, 107, 768, 768) spacing: [5.         0.51822919 0.51822919] 




 Task508_AMOS_kidney
number of threads:  (8, 8) 

Are we using the nonzero mask for normalization? OrderedDict([(0, False)])
the median shape of the dataset is  [139.7515528  263.90122779 263.90122779]
the max shape in the dataset is  [166.14906832 270.37037037 270.37037037]
the min shape in the dataset is  [121.11801242 245.67902176 245.67902176]
we don't want feature maps smaller than  4  in the bottleneck
t

In [ ]:
!nnUNet_plan_and_preprocess -t 507 -pl2d None -pl3d ExperimentPlanner3D_v21_customTargetSpacing_kits19 -no_pp

case_00000
case_00001
case_00002
case_00003
case_00004
before crop: (1, 64, 512, 512) after crop: (1, 64, 512, 512) spacing: [4.        0.9765625 0.9765625] 

before crop: (1, 261, 512, 512) after crop: (1, 261, 512, 512) spacing: [1.         0.93945312 0.93945312] 

before crop: (1, 270, 512, 512) after crop: (1, 270, 512, 512) spacing: [1.         0.85546875 0.85546875] 

before crop: (1, 611, 512, 512) after crop: (1, 611, 512, 512) spacing: [0.5        0.91992188 0.91992188] 

before crop: (1, 602, 512, 512) after crop: (1, 602, 512, 512) spacing: [0.5        0.79882812 0.79882812] 




 Task507_KiTS_kidney
number of threads:  (8, 8) 

Are we using the nonzero mask for normalization? OrderedDict([(0, False)])
the median shape of the dataset is  [ 83.85093168 290.74074074 290.74074074]
the max shape in the dataset is  [ 94.8757764  308.64197531 308.64197531]
the min shape in the dataset is  [ 79.50310559 252.4691358  252.4691358 ]
we don't want feature maps smaller than  4  in the b

In [ ]:
!python dataset_conversion/kits_convert_label.py intensity

before changing...
mean: 105.65725708007812
std: 74.52706146240234
percentile_99_5: 251.0
percentile_00_5: -88.0
after changing...
mean: 32.562679290771484
std: 131.16937255859375
percentile_99_5: 252.0
percentile_00_5: -979.0


In [ ]:
!nnUNet_plan_and_preprocess -t 507 -pl2d None -pl3d ExperimentPlanner3D_v21_customTargetSpacing_kits19 -no_plan

case_00000
case_00001
case_00002
case_00003
case_00004



 Task507_KiTS_kidney
number of threads:  (8, 8) 

Initializing to run preprocessing
npz folder: /home/vmagent/app/dataset/nnUNet_raw_data_base/nnUNet_cropped_data/Task507_KiTS_kidney
output_folder: /home/vmagent/app/dataset/nnUNet_preprocessed/Task507_KiTS_kidney
separate z, order in z is 0 order inplane is 3
no separate z, order 3
no separate z, order 3
separate z, order in z is 0 order inplane is 1
before: {'spacing': array([4.       , 0.9765625, 0.9765625]), 'spacing_transposed': array([4.       , 0.9765625, 0.9765625]), 'data.shape (data is transposed)': (1, 64, 512, 512)} 
after:  {'spacing': array([3.22, 1.62, 1.62]), 'data.shape (data is resampled)': (1, 80, 309, 309)} 

1 10000
saving:  /home/vmagent/app/dataset/nnUNet_preprocessed/Task507_KiTS_kidney/nnUNetData_plans_v2.1_trgSp_kits19_stage0/case_00004.npz
no separate z, order 3
no separate z, order 3
no separate z, order 1
no separate z, order 1
before: {'spacing': arr

# 5. Launch training

## 5.1 Pre-train Target Domain

- We will first pre-train model in AMOS dataset, and use this pre-trained model later for prameter initialization for domain adaptation
- We use [3D-UNet](https://arxiv.org/abs/1606.06650) to train the model
- *For demostration, we only train 1 epochs:*

In [ ]:
!nnUNet_train 3d_fullres nnUNetTrainerV2 508 1 --epochs 1 -p nnUNetPlansv2.1_trgSp_kits19 --disable_postprocessing_on_folds

###############################################
I am running the following nnUNet: 3d_fullres
My trainer class is:  <class 'nnunet.training.network_training.nnUNetTrainerV2.nnUNetTrainerV2'>
For that I will be using the following configuration:
num_classes:  15
modalities:  {0: 'CT'}
use_mask_for_norm OrderedDict([(0, False)])
keep_only_largest_region None
min_region_size_per_class None
min_size_per_class None
normalization_schemes OrderedDict([(0, 'CT')])
stages...

stage:  0
{'batch_size': 2, 'num_pool_per_axis': [4, 5, 5], 'patch_size': array([ 80, 160, 160]), 'median_patient_size_in_voxels': array([140, 264, 264]), 'current_spacing': array([3.22, 1.62, 1.62]), 'original_spacing': array([3.22, 1.62, 1.62]), 'do_dummy_2D_data_aug': False, 'pool_op_kernel_sizes': [[2, 2, 2], [2, 2, 2], [2, 2, 2], [2, 2, 2], [1, 2, 2]], 'conv_kernel_sizes': [[3, 3, 3], [3, 3, 3], [3, 3, 3], [3, 3, 3], [3, 3, 3], [3, 3, 3]]}

I am using stage 0 from these plans
I am using sample dice + CE loss

I am usi

2022-10-14 06:36:18.362553: train loss : 0.5081
2022-10-14 06:39:48.485496: validation loss: 0.3403
2022-10-14 06:39:48.486991: Average global foreground Dice: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
2022-10-14 06:39:48.487164: (interpret this as an estimate for the Dice of the different classes. This is not exact.)
2022-10-14 06:39:48.979372: lr: 0.009991
2022-10-14 06:39:48.979655: This epoch took 3321.872497 s

2022-10-14 06:39:48.980681: saving checkpoint...
2022-10-14 06:39:49.231418: done, saving took 0.25 seconds
amos_0007 (2, 166, 246, 246)
debug: mirroring True mirror_axes (0, 1, 2)
step_size: 0.5
do mirror: True
data shape: (1, 166, 246, 246)
patch size: [ 80 160 160]
steps (x, y, and z): [[0, 29, 57, 86], [0, 43, 86], [0, 43, 86]]
number of tiles: 36
computing Gaussian
done
prediction done
force_separate_z: None interpolation order: 1
separate z: True lowres axis [0]
separate z, order in z is 0 order inplane is 1
^C
Process ForkPoolWorker-

## 5.2 Domain Adaption from AMOS to KiTS

- We use a DANN-like model architecture, the DANN algorithm is illustrated as follows:
![dann](../imgs/dann.png)

- Now we use Model Adapter API to transfer knowledge from AMOS dataset to KiTS dataset

```python
from e2eAIOK.ModelAdapter.backbone.unet.generic_UNet_DA import Generic_UNet_DA
from e2eAIOK.ModelAdapter.engine_core.adapter.adversarial.DA_Loss import CACDomainAdversarialLoss
from e2eAIOK.ModelAdapter.engine_core.transferrable_model import make_transferrable_with_domain_adaption

backbone = Generic_UNet_DA(
    self.threeD, self.num_input_channels, 
    self.base_num_features, self.num_classes,         
    self.conv_per_stage, self.net_num_pool_op_kernel_sizes, 
    self.net_conv_kernel_sizes
)

adv_kwargs = {
    'input_channels': backbone.encoder_channels,
    'threeD': self.threeD,
    'pool_op_kernel_sizes': self.net_num_pool_op_kernel_sizes,
    'loss_weight': self.loss_weights[2:]
}
cac_domain_adv = CACDomainAdversarialLoss(**adv_kwargs)

self.network = make_transferrable_with_domain_adaption(
    backbone, None, cac_domain_adv, 
    False, self.source_loss_weight, 1.0)
```

- After using `make_transferrable_with_domain_adaption`, we got an adapted model, we use this model for further training. We use the following command to start training,

In [ ]:
!nnUNet_train_da 3d_fullres nnUNetTrainer_DA_V2 508 507 1 \
    -p nnUNetPlansv2.1_trgSp_kits19 \
    -sp nnUNetPlansv2.1_trgSp_kits19 \
    --epochs 1 --loss_weights 1 0 1 0 0 \
    -pretrained_weights /home/vmagent/app/dataset/nnUNet_trained_models/nnUNet/3d_fullres/Task508_AMOS_kidney/nnUNetTrainerV2__nnUNetPlansv2.1_trgSp_kits19/fold_1/model_final_checkpoint.model 


###############################################
For that I will be using the following source data configuration:
num_classes:  15
modalities:  {0: 'CT'}
use_mask_for_norm OrderedDict([(0, False)])
keep_only_largest_region None
min_region_size_per_class None
min_size_per_class None
normalization_schemes OrderedDict([(0, 'CT')])
stages...

stage:  0
{'batch_size': 2, 'num_pool_per_axis': [4, 5, 5], 'patch_size': array([ 80, 160, 160]), 'median_patient_size_in_voxels': array([140, 264, 264]), 'current_spacing': array([3.22, 1.62, 1.62]), 'original_spacing': array([3.22, 1.62, 1.62]), 'do_dummy_2D_data_aug': False, 'pool_op_kernel_sizes': [[2, 2, 2], [2, 2, 2], [2, 2, 2], [2, 2, 2], [1, 2, 2]], 'conv_kernel_sizes': [[3, 3, 3], [3, 3, 3], [3, 3, 3], [3, 3, 3], [3, 3, 3], [3, 3, 3]]}


I am using source data from this folder:  /home/vmagent/app/dataset/nnUNet_preprocessed/Task508_AMOS_kidney/nnUNetData_plans_v2.1_trgSp_kits19
###############################################
#################

2022-10-14 08:03:19.386149: train loss : 1.0404
2022-10-14 08:05:40.979750: validation loss: 0.0087
2022-10-14 08:05:40.981636: Average global foreground Dice: [0.0]
2022-10-14 08:05:40.981980: (interpret this as an estimate for the Dice of the different classes. This is not exact.)
2022-10-14 08:05:41.279021: lr index 0: 0.001982
2022-10-14 08:05:41.279208: lr index 1: 0.00991
2022-10-14 08:05:41.279391: This epoch took 3509.259070 s

2022-10-14 08:05:41.280476: saving checkpoint...
2022-10-14 08:05:41.633380: done, saving took 0.35 seconds
case_00004 (2, 80, 309, 309)
debug: mirroring True mirror_axes (0, 1, 2)
step_size: 0.5
do mirror: True
data shape: (1, 80, 309, 309)
patch size: [ 80 160 160]
steps (x, y, and z): [[0], [0, 74, 149], [0, 74, 149]]
number of tiles: 9
computing Gaussian
done
prediction done
force_separate_z: None interpolation order: 1
separate z: True lowres axis [0]
separate z, order in z is 0 order inplane is 1
2022-10-14 08:07:06.740799: finished prediction
2022

- Notice: 
    - we donot use **any label** from target domain KiTS, we only use label from source domain AMOS for training
    - *For demostration, we only train 1 epochs:*

# 6. Launch Inference & Evaluation

## 6.1 Inference on KiTS Dataset with Adapted Model

- Now we use the adapted model trained in last section to perferm inference on KiTS dataset

- We use following command for perform inference and evaluation, you can find your predictions in `${nnUNet_raw_data_base}/nnUNet_raw_data/Task507_KiTS_kidney/predict/`

In [ ]:
# # inference
time nnUNet_predict \
    -i ${nnUNet_raw_data_base}/nnUNet_raw_data/Task507_KiTS_kidney/testTr/ \
    -o ${nnUNet_raw_data_base}/nnUNet_raw_data/Task507_KiTS_kidney/predict/ \
    -f 1 \
    -t 507 -m 3d_fullres -p nnUNetPlansv2.1_trgSp_kits19 \
    --disable_tta \
    -tr nnUNetTrainer_DA_V2 \
    --overwrite_existing \
    --disable_mixed_precision 

using model stored in  /home/vmagent/app/dataset/nnUNet_trained_models/nnUNet/3d_fullres/Task507_KiTS_kidney/nnUNetTrainer_DA_V2__nnUNetPlansv2.1_trgSp_kits19
This model expects 1 input modalities for each image
Found 5 unique case ids, here are some examples: ['case_00000' 'case_00000' 'case_00003' 'case_00003' 'case_00002']
If they don't look right, make sure to double check your filenames. They must end with _0000.nii.gz etc
number of cases: 5
number of cases that still need to be predicted: 5
emptying cuda cache
loading parameters for folds, [1]
using the following model files:  ['/home/vmagent/app/dataset/nnUNet_trained_models/nnUNet/3d_fullres/Task507_KiTS_kidney/nnUNetTrainer_DA_V2__nnUNetPlansv2.1_trgSp_kits19/fold_1/model_final_checkpoint.model']
starting preprocessing generator
starting prediction...
preprocessing /home/vmagent/app/dataset/prediction/case_00000.nii.gz
using preprocessor GenericPreprocessor
preprocessing /home/vmagent/app/dataset/prediction/case_00001.nii.gz
u

## 6.2 Evaluate the Prediction on KiTS Using the Given Label

- Note while evaluating: 
    - The label is not used in training, it is only used in this evaluation step
    - In practical, if you donnot have any label, you can just skip this step

In [ ]:
# # evaluate with given label    
nnUNet_evaluate_folder \
    -ref ${nnUNet_raw_data_base}/nnUNet_raw_data/Task507_KiTS_kidney/labelsTr \
    -pred ${nnUNet_raw_data_base}/nnUNet_raw_data/Task507_KiTS_kidney/predict \
    -l 1 \
    --common

/work/AIDK/AIDK/TransferLearningKit/src/task/medical_segmentation/nnunet/evaluation/evaluator.py:381: RuntimeWarning: Mean of empty slice
  all_scores["mean"][label][score] = float(np.nanmean(all_scores["mean"][label][score]))
0.0


# 7. Visualization of Data and Segmentations

- Download files from server:
   - Images from: ```${nnUNet_raw_data_base}/nnUNet_raw_data/Task507_KiTS_kidney/imagesTr/```
   - Segmentations from: ```${nnUNet_raw_data_base}/nnUNet_raw_data/Task507_KiTS_kidney/labelsTr/```
   - predictions from: ```${nnUNet_raw_data_base}/nnUNet_raw_data/Task507_KiTS_kidney/predict/```
- After downloading these files you can visualize them with any volumetric visualization program.
For this we would advise to use [MITK](https://www.mitk.org/wiki/The_Medical_Imaging_Interaction_Toolkit_(MITK)) which already has some great [tutorials](https://www.mitk.org/wiki/Tutorials). 
    - If you have not already downloaded it, here is the [MITK Download Link](https://www.mitk.org/wiki/Downloads) 
- Here is a demostration of visualization result from MITK on KiTS dataset
![KiTS_visualization](../imgs/KiTS_visualization.png)

